## Error Back Propagation

In [25]:
from math import exp
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from random import random,seed,randrange

In [27]:
dataset=pd.read_csv('Housing.csv')
dataset=dataset.drop(columns=['id','recroom','fullbase','gashw','airco'])

# train set
train=dataset.values.tolist()
#test set
test=dataset.drop(columns=['prefarea']).values.tolist()

## 1. Initialize Neural Network

In [44]:
# Initialize a network
def neural_network(inputs,hidden,outputs):
    seed(1)
    network = list()
    hidden_layer = [{'weights':[random() for i in range(inputs + 1)]} for i in range(hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights':[random() for i in range(hidden + 1)]} for i in range(outputs)]
    network.append(output_layer)
    return network

## 2. Feed Forward

In [31]:
# weighted sum
def weighted_sum(weights, inputs):
    summation = weights[-1]
    for i in range(len(weights)-1):
        summation += weights[i] * inputs[i]
    return summation      

In [32]:
#sigmoid fn
def sigmoid(x):
    return 1.0 / (1.0 + exp(-x))

In [33]:
# Feed forward
def feed_forward(network, inputs):
    for layer in network:
        new_inputs = []
        for neuron in layer:
            summation = weighted_sum(neuron['weights'], inputs)
            neuron['output'] = sigmoid(summation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs

## 3. Back Propagation

In [34]:
#sigmoid derivative
def sigmoid_derivative(x):
    return x*(1-x)

In [68]:
# Backpropagate error
def backward_propagate(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * sigmoid_derivative(neuron['output'])
    return errors

## 4. Train Network

In [69]:
# Update network weights with error
def update_weights(network, row, learning_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += learning_rate*neuron['delta']*inputs[j]
            neuron['weights'][-1] += learning_rate*neuron['delta']

In [90]:
# Train network
def train_network(network, train, learning_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        for row in train:
            outputs = feed_forward(network, row)
            expected = [0 for i in range(n_outputs)]
            #print(row[-1])
            expected[int(row[-1])] = 1
            errors=backward_propagate(network, expected)
            update_weights(network, row, learning_rate)

In [88]:
#predict
def predict(network, row):
    outputs = feed_forward(network, row)
    return outputs.index(max(outputs))

## Main Function

In [79]:
# Backpropagation Algorithm
def back_propagation(train, test, learning_rate, n_epoch, n_hidden):
    n_inputs = len(train[0]) - 1
    n_outputs = len(set([row[-1] for row in train]))
    network = neural_network(n_inputs, n_hidden, n_outputs)
    train_network(network, train, learning_rate, n_epoch, n_outputs)
    predictions = list()
    for row in test:
        prediction = predict(network, row)
        predictions.append(prediction)
    return(predictions)

## Accuracy of Algorithm

In [80]:
# evaluate accuracy
def evaluate_algo(predict,test):
    count=0
    for i in range(len(test)):
        if(predict[i]==test[i]):
            count+=1
    accuracy=(count/len(predict))*100
    return accuracy

In [91]:
predict=back_propagation(train,test,0.1,500,5)
actual_op=dataset['prefarea'].values.tolist()

In [92]:
print("Accuracy =",evaluate_algo(predict,actual_op),"%")

Accuracy = 76.55677655677655 %
